In [2]:
import re, json, requests
import pandas as pd

# Samples

In [3]:
acc = pd.read_csv('../data/glycine_max_sra.csv')
acc = acc.loc[acc['assay_type'] == 'RNA-Seq']
acc = acc.set_index('biosample')
acc

,acc,assay_type,center_name,consent,experiment,sample_name,instrument,librarylayout,libraryselection,librarysource,...,ena_first_public_run,ena_last_update_run,sample_name_sam,datastore_filetype,datastore_provider,datastore_region,attributes,jattr,run_file_version,random
biosample,,,,,,,,,,,,,,,,,,,,,
SAMD00235537,DRR237204,RNA-Seq,RIKEN_CSRS,public,DRX227388,SAMD00235537,Illumina HiSeq 2500,PAIRED,PolyA,TRANSCRIPTOMIC,...,NaN,NaN,[Aspergillus_oryzae_DAS6],"[sra, run.zq]","[s3, ncbi, gs]","[ncbi.public, s3.us-east-1, gs.us-east1]","[{k=bases, v=4673307400}, {k=bytes, v=22975060...","{'bases': '4673307400', 'bytes': '2297506096',...",1.0,0.999995
SAMN30889879,SRR21660183,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659913,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[sra, run.zq, fastq]","[ncbi, gs, s3]","[ncbi.public, s3.us-east-1, gs.us-east1]","[{k=bases, v=6300717300}, {k=bytes, v=20041210...","{'bases': '6300717300', 'bytes': '2004121032',...",1.0,0.999912
SAMN09513405,SRR7447776,RNA-Seq,THE OHIO STATE UNIVERSITY,public,SRX4318240,SL209648,Illumina HiSeq 2500,PAIRED,PolyA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, run.zq, sra]","[gs, ncbi, s3]","[gs.us-east1, ncbi.public, s3.us-east-1]","[{k=bases, v=634701400}, {k=bytes, v=238170360...","{'bases': '634701400', 'bytes': '238170360', '...",1.0,0.999817
SAMN09078292,SRR7124263,RNA-Seq,GEO,public,SRX4045331,GSM3132253,Illumina HiSeq 2000,SINGLE,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, run.zq, sra]","[gs, ncbi, s3]","[gs.us-east1, ncbi.public, s3.us-east-1]","[{k=geo_accession_exp, v=GSM3132253}, {k=bases...","{'geo_accession_exp': 'GSM3132253', 'bases': '...",1.0,0.999636
SAMN36760700,SRR25459671,RNA-Seq,SOYBEAN,public,SRX21192507,R89-2-1,Illumina HiSeq 4000,PAIRED,PCR,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[sra, run.zq, fastq]","[s3, gs, ncbi]","[ncbi.public, gs.us-east1, s3.us-east-1]","[{k=bases, v=6132149664}, {k=bytes, v=19279351...","{'bases': '6132149664', 'bytes': '1927935140',...",1.0,0.999558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN21359677,SRR15899288,RNA-Seq,USDA,public,SRX12189762,G3_PI578366_Leaf_Suf_Rep1,Illumina HiSeq 2500,SINGLE,unspecified,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[run.zq, sra]","[s3, gs, ncbi]","[ncbi.public, gs.us-east1, s3.us-east-1]","[{k=assemblyname, v=GCA_000004515.4}, {k=bases...","{'assemblyname': 'GCA_000004515.4', 'bases': '...",1.0,0.000509
SAMN20971793,SRR15610895,RNA-Seq,GEO,public,SRX11908208,GSM5536784,AB SOLiD System 3.0,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, sra]","[s3, gs, ncbi]","[s3.us-east-1, gs.us-east1, ncbi.public]","[{k=geo_accession_exp, v=GSM5536784}, {k=bases...","{'geo_accession_exp': 'GSM5536784', 'bases': '...",1.0,0.000492
SAMN41726398,SRR29315715,RNA-Seq,NORTHEAST AGRICULTURAL UNIVERSITY,public,SRX24832506,HPHO_MM_2,Illumina HiSeq 2000,PAIRED,RT-PCR,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[sra, run.zq, fastq]","[gs, ncbi, s3]","[ncbi.public, s3.us-east-1, gs.us-east1]","[{k=bases, v=14794300800}, {k=bytes, v=5886791...","{'bases': '14794300800', 'bytes': '5886791527'...",1.0,0.000401


In [4]:
acc.loc['SAMN30889879']

,acc,assay_type,center_name,consent,experiment,sample_name,instrument,librarylayout,libraryselection,librarysource,...,ena_first_public_run,ena_last_update_run,sample_name_sam,datastore_filetype,datastore_provider,datastore_region,attributes,jattr,run_file_version,random
biosample,,,,,,,,,,,,,,,,,,,,,
SAMN30889879,SRR21660183,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659913,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[sra, run.zq, fastq]","[ncbi, gs, s3]","[ncbi.public, s3.us-east-1, gs.us-east1]","[{k=bases, v=6300717300}, {k=bytes, v=20041210...","{'bases': '6300717300', 'bytes': '2004121032',...",1.0,0.999912
SAMN30889879,SRR21660178,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659917,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[run.zq, fastq, sra]","[s3, ncbi, gs]","[s3.us-east-1, gs.us-east1, ncbi.public]","[{k=bases, v=6324660600}, {k=bytes, v=19900827...","{'bases': '6324660600', 'bytes': '1990082748',...",1.0,0.900161
SAMN30889879,SRR21660162,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659933,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, sra, run.zq]","[s3, gs, ncbi]","[gs.us-east1, ncbi.public, s3.us-east-1]","[{k=bases, v=7010192700}, {k=bytes, v=22213398...","{'bases': '7010192700', 'bytes': '2221339825',...",1.0,0.863402
SAMN30889879,SRR21660160,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659935,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, run.zq, sra]","[gs, s3, ncbi]","[s3.us-east-1, gs.us-east1, ncbi.public]","[{k=bases, v=7022437800}, {k=bytes, v=22099777...","{'bases': '7022437800', 'bytes': '2209977773',...",1.0,0.841321
SAMN30889879,SRR21660173,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659923,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[run.zq, sra, fastq]","[ncbi, gs, s3]","[gs.us-east1, ncbi.public, s3.us-east-1]","[{k=bases, v=5820658200}, {k=bytes, v=18284805...","{'bases': '5820658200', 'bytes': '1828480526',...",1.0,0.830372
SAMN30889879,SRR21660169,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659926,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, run.zq, sra]","[s3, ncbi, gs]","[gs.us-east1, s3.us-east-1, ncbi.public]","[{k=bases, v=7064327100}, {k=bytes, v=22300297...","{'bases': '7064327100', 'bytes': '2230029707',...",1.0,0.484343
SAMN30889879,SRR21660172,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659924,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[sra, run.zq, fastq]","[ncbi, gs, s3]","[s3.us-east-1, gs.us-east1, ncbi.public]","[{k=bases, v=7099019100}, {k=bytes, v=22390340...","{'bases': '7099019100', 'bytes': '2239034095',...",1.0,0.353955
SAMN30889879,SRR21660170,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659925,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[fastq, sra, run.zq]","[ncbi, gs, s3]","[gs.us-east1, s3.us-east-1, ncbi.public]","[{k=bases, v=6311261400}, {k=bytes, v=20130125...","{'bases': '6311261400', 'bytes': '2013012588',...",1.0,0.303277
SAMN30889879,SRR21660176,RNA-Seq,INNER MONGOLIA ACADEMY OF AGRICULTURAL & ANIMA...,public,SRX17659919,"CK (1,2,3,4)",Illumina NovaSeq 6000,PAIRED,cDNA,TRANSCRIPTOMIC,...,NaN,NaN,NaN,"[run.zq, fastq, sra]","[gs, s3, ncbi]","[gs.us-east1, ncbi.public, s3.us-east-1]","[{k=bases, v=6582839400}, {k=bytes, v=21091275...","{'bases': '6582839400', 'bytes': '2109127502',...",1.0,0.172338


In [5]:
attr = pd.DataFrame.from_records(acc['jattr'].apply(eval), index = acc.index)
attr

/tmp/ipykernel_68288/1648790811.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  attr = pd.DataFrame.from_records(acc['jattr'].apply(eval), index = acc.index)


,bases,bytes,run_file_create_date,age_sam,treatment_sam_ss_dpl55,primary_search,cultivar_sam,dev_stage_sam,ecotype_sam,isolate_sam_ss_dpl100,...,quality_scoring_system_run,accession_ecotype_sam,tissue_and_developmental_stage_sam,experimental_factor__1_4_exp,concentration_sam,cultivation_pattren_sam,er_gfp_sam,subsrc_note_sam_s_dpl392,authors_sam,env_biome_sam
biosample,,,,,,,,,,,,,,,,,,,,,
SAMD00235537,4673307400,2297506096,2020-12-24T20:19:00.000Z,DAS6,Aspergillus oryzae,SAMD00235537,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAMN30889879,6300717300,2004121032,2022-09-21T14:15:00.000Z,missing,NaN,bp0,missing,seedling stage,missing,B sufficient,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAMN09513405,634701400,238170360,2018-06-28T09:53:00.000Z,24 hai,Inoculated,bp0,12279,NaN,Conrad x Sloan,P. sojae 1.S.1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAMN09078292,2489734296,892297781,2018-05-07T10:10:00.000Z,NaN,NaN,SRX4045331,PI641156 NC-Raleigh,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAMN36760700,6132149664,1927935140,2023-07-31T00:31:00.000Z,NaN,NaN,bp0,NaN,R89-2-1 at low temperature treatment for 12 hours,NaN,R89-2 line repeat 1 in CSSL population,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN21359677,696270,579898,2021-09-14T19:37:00.000Z,60 minutes after iron stress,Sufficient iron,SRX12189762,Hong hu liu yue bao,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAMN20971793,759144485,854616392,2021-12-06T23:47:00.000Z,NaN,NaN,SRX11908208,Soybean cultivar Williams,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SAMN41726398,14794300800,5886791527,2024-06-07T04:38:00.000Z,NaN,NaN,bp0,NaN,Mid Seed Maturity,NaN,HPHO_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
attr.isna().mean().sort_values()

primary_search                         0.000000
bases                                  0.000919
bytes                                  0.000919
run_file_create_date                   0.001609
tissue_sam_ss_dpl145                   0.134048
                                         ...   
authors_sam                            0.999847
tissue_and_developemental_stage_sam    0.999847
concentration_sam                      0.999923
cultivation_pattren_sam                0.999923
env_biome_sam                          0.999923
Length: 232, dtype: float64

In [7]:
acc.to_csv('helper/acc.csv')
attr.to_csv('helper/attr.csv')

In [8]:
def get_biosample(biosamp_id):
    api_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=biosample&id={biosamp_id}&rettype=full&retmode=text"
    res = requests.get(api_url)
    return res.text

def metadata_json(metadata):
    res = {}
    res['title'] = metadata.split('\n')[0].replace('1: ', '')
    identifiers = re.search(r'Identifiers: (.+)', metadata)
    if identifiers:
        res['Identifiers'] = {k.strip(): v.strip() for k, v in 
                                [item.split(':') if ':' in item else (item, "")
                                    for item in identifiers.group(1).split(';')]}
    organism = re.search(r'Organism: (.+)', metadata)
    if organism:
        res['Organism'] = organism.group(1).strip()
    attributes = re.findall(r'/(\w+)="([^"]+)"', metadata)
    if attributes:
        res['Attributes'] = {k: v for k, v in attributes}
    accession = re.search(r'Accession: (\w+)', metadata)
    if accession:
        res['Accession'] = accession.group(1)
    id_match = re.search(r'ID: (\d+)', metadata)
    if id_match:
        res['ID'] = id_match.group(1)
    
    return json.dumps(res)

In [9]:
try:
    biosamp = pd.read_csv('helper/biosamp.csv', index_col=0)
except:
    biosamp = pd.DataFrame(acc['biosample'])
    biosamp['metadata'] = acc['biosample'].apply(get_biosample)
    biosamp['jmeta'] = biosamp['metadata'].apply(metadata_json)
    biosamp = biosamp.set_index('biosample')
    biosamp = pd.DataFrame(biosamp)
    biosamp.to_csv('helper/biosamp.csv')

In [11]:
jmeta = pd.DataFrame.from_records(biosamp['metadata'].apply(metadata_json).apply(json.loads), index = biosamp.index)
jmeta

/tmp/ipykernel_68288/1722256263.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  jmeta = pd.DataFrame.from_records(biosamp['metadata'].apply(metadata_json).apply(json.loads), index = biosamp.index)


,title,Identifiers,Organism,Attributes,Accession,ID
biosample,,,,,,
SAMD00235537,Oryza sativa Japonica Group NG8334,"{'BioSample': 'SAMN00235537', 'GSS': 'LIBGSS_0...",Oryza sativa Japonica Group,"{'strain': 'NG8334', 'cultivar': 'Nipponbare'}",SAMN00235537,235537
SAMN30889879,Plant sample from Glycine max,"{'BioSample': 'SAMN30889879', 'Sample name': '...",Glycine max,"{'isolate': 'B sufficient', 'cultivar': 'missi...",SAMN30889879,30889879
SAMN09513405,Plant sample from Glycine max,"{'BioSample': 'SAMN09513405', 'Sample name': '...",Glycine max,"{'isolate': 'P. sojae 1.S.1.1', 'cultivar': '1...",SAMN09513405,9513405
SAMN09078292,mRNA gmax PI641156 NC-Raleigh,"{'BioSample': 'SAMN09078292', 'SRA': 'SRS32627...",Glycine max,"{'cultivar': 'PI641156 NC-Raleigh', 'tissue': ...",SAMN09078292,9078292
SAMN36760700,Plant sample from Glycine max,"{'BioSample': 'SAMN36760700', 'Sample name': '...",Glycine max,{'isolate': 'R89-2 line repeat 1 in CSSL popul...,SAMN36760700,36760700
...,...,...,...,...,...,...
SAMN25827181,12dpi-2C-Replicate2,"{'BioSample': 'SAMN25827181', 'Sample name': '...",Glycine max,"{'cultivar': 'Williams 82', 'age': '12 days po...",SAMN25827181,25827181
SAMN20971793,Mock-inoculated Williams rep 2 [WM-2a],"{'BioSample': 'SAMN20971793', 'SRA': 'SRS99190...",Glycine max,{'cultivar': 'Soybean cultivar Williams'},SAMN20971793,20971793
SAMN41726398,Plant sample from Glycine max,"{'BioSample': 'SAMN41726398', 'Sample name': '...",Glycine max,"{'isolate': 'HPHO_2', 'tissue': 'seed'}",SAMN41726398,41726398


In [12]:
jattr = pd.DataFrame.from_records(jmeta['Attributes'].dropna(), index = jmeta['Attributes'].dropna().index)
jident = pd.DataFrame.from_records(jmeta['Identifiers'].dropna(), index = jmeta['Identifiers'].dropna().index).drop('BioSample', axis=1)

/tmp/ipykernel_68288/987814684.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  jattr = pd.DataFrame.from_records(jmeta['Attributes'].dropna(), index = jmeta['Attributes'].dropna().index)
/tmp/ipykernel_68288/987814684.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  jident = pd.DataFrame.from_records(jmeta['Identifiers'].dropna(), index = jmeta['Identifiers'].dropna().index).drop('BioSample', axis=1)


In [19]:
biometa = pd.DataFrame(jmeta['title']).join(jattr, how='left', rsuffix='_jattr').join(jident, how='left', rsuffix='_jident')

,title,strain,cultivar,isolate,ecotype,age,tissue,disease,population,treatment,...,HostIndividual,barcodesequence,scientific_name,GSS,Sample name,SRA,GEO,DOE Joint Genome Institute,EST,SCN
biosample,,,,,,,,,,,,,,,,,,,,,
SAMD00235537,Oryza sativa Japonica Group NG8334,NG8334,Nipponbare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,LIBGSS_024689,NaN,NaN,NaN,NaN,NaN,NaN
SAMN30889879,Plant sample from Glycine max,NaN,missing,B sufficient,missing,missing,Root,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"CK (1,2,3,4)",SRS15192176,NaN,NaN,NaN,NaN
SAMN09513405,Plant sample from Glycine max,NaN,12279,P. sojae 1.S.1.1,Conrad x Sloan,24 hai,inoculation site: ~2 cm below the crown,Phytophthora root and stem rot,F9:11 Recombinant inbred line,Inoculated,...,NaN,NaN,NaN,NaN,SL209648,SRS3480825,NaN,NaN,NaN,NaN
SAMN09078292,mRNA gmax PI641156 NC-Raleigh,NaN,PI641156 NC-Raleigh,NaN,NaN,NaN,leaf,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SRS3262752,GSM3132253,NaN,NaN,NaN
SAMN36760700,Plant sample from Glycine max,NaN,NaN,R89-2 line repeat 1 in CSSL population,NaN,NaN,Embryo,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,R89-2-1,SRS18451534,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAMN25827181,12dpi-2C-Replicate2,NaN,Williams 82,NaN,NaN,12 days post inoculation,root nodules,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,II-Early-12dpi-2C_rep2,SRS11952324,NaN,NaN,NaN,NaN
SAMN20971793,Mock-inoculated Williams rep 2 [WM-2a],NaN,Soybean cultivar Williams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SRS9919099,GSM5536784,NaN,NaN,NaN
SAMN41726398,Plant sample from Glycine max,NaN,NaN,HPHO_2,NaN,NaN,seed,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,HPHO_MM_2,SRS21544452,NaN,NaN,NaN,NaN


In [ ]:
biometa.to_csv('helper/biometa.csv')

# Unified

In [ ]:
acc = pd.read_csv('helper/acc.csv')
attr = pd.read_csv('helper/attr.csv')
biometa = pd.read_csv('helper/biometa.csv')

In [ ]:
acc = acc.drop_duplicates(keep='first', subset='biosample').set_index('biosample')
attr = attr.drop_duplicates(keep='first', subset='biosample').set_index('biosample')
biometa = biometa.drop_duplicates(keep='first', subset='biosample').set_index('biosample')

In [ ]:
joined = acc.join(attr, how='outer', rsuffix='_attr').join(jmeta, how='outer', rsuffix='_jmeta')

In [ ]:
# joined = pd.concat([acc, attr, jmeta], axis=1)
joined.to_csv('joined.csv')
joined

# Bioprojects

In [ ]:
def get_bioproject(bioproj_id):
    api_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=bioproject&id={bioproj_id}"
    res = requests.get(api_url)
    return res.text

In [ ]:
venancio = pd.read_json('venancio.json')
venancio.columns = ['bioproject', 'n', 'part', 'title', 'abstract', 'doi', 'pmid']
venancio

In [ ]:
projects = pd.DataFrame({'bioproject': joined.bioproject.unique()})
projects

In [ ]:
projects = projects.join(venancio.set_index('bioproject'), rsuffix='_venancio')
projects

In [ ]:
projects['xml'] = projects['bioproject'].apply(get_bioproject)

In [ ]:
projects